In [ ]:

Проект: Анализ резюме из HeadHunter

# Данная библиотека импортируется ИСКЛЮЧИТЕЛЬНО для возможности отображения графиков через
# https://nbviewer.org/ Jupyter Notebook. Для действий над данными, НЕ ИСПОЛЬЗУЕТСЯ!!!
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Данная библиотека импортируется ИСКЛЮЧИТЕЛЬНО для возможности отображения графиков через
# https://nbviewer.org/ Jupyter Notebook. Для действий над данными, НЕ ИСПОЛЬЗУЕТСЯ!!!
import plotly.io as pio

Исследование структуры данных

    Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла.

# читаем данные из файла data/dst-3.0_16_1_hh_database.csv.
# принимаем во внимание, что данные по признакам разделены ';', об этом сообщаем функции
# read_csv через параметр sep = ';'
data = pd.read_csv('data/dst-3.0_16_1_hh_database.csv', sep=';')

# оставим оригинальный датасет без изменений, и будем работать с его точной копией
# получим это через метод copy
data_hh = data.copy()

    Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

# Выведем несколько последних строк, чтобы убедиться что данные прочлись и 
# разделились по признакам корректно. И вкратце ознакомится с структурой и 
# организацией данных 
# Используем для этого метод tail, где параметр в скобках кол-во строк для вывода в конце

display(data_hh.tail(3))

# Также могли бы использовать и такой подход для получения результата 
# display(data_hh.loc[data_hh.index[-3:]])
# получение диапазона индекса data_hh.index[-3:] - > вывод строк относящихся к данному диапазону индекса,
# через data_hh.loc[]



In [ ]:


    Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

    Обратите внимание на информацию о числе непустых значений.

# Выведем информацию о признаках, для этого использум метод info
data_hh.info()



   5 Выведите основную статистическую информацию о столбцах.

# Для вывода статистики по столбцам, используем метод describe.
# Учитывая, что все столбцы у нас нас типа object, то мы не получим такую информацию, 
# как среднее значение, стандартное отклоненение и т.д, то что касается числовых данных
data_hh.describe()



In [ ]:

Преобразование данных

    Начнем с простого - с признака "Образование и ВУЗ". Его текущий формат это: <Уровень образования год выпуска ВУЗ специальность...>. Например:

    Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
    Неоконченное высшее образование 2000 Балтийская государственная академия рыбопромыслового флота…

Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак "Образование", который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце "Образование и ВУЗ". Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

Совет: проверяйте полученные категории, например, с помощью метода unique()

def extract_education(full_education)->str:
  """Функция выявляющая из полной информации об обучении
    уровень образования, из строки формата 
    <Уровень образования год выпуска ВУЗ специальность...>
    например: 
    На входе: Неоконченное высшее образование 2000 Балтийская государственная академия рыбопромыслового флота
    На выходе:неоконченное высшее 
  
  Args:
      full_education (str): Получаем всю строку с описанием образования

  Returns:
      str: Возвращаем уровень образования
  """
  ed_arr = full_education.split(' ')
  # исходя из подсказки, что образование описывается в первом или первыми 2-мя словами,
  # то преобразуем всю информацию об образовании в list и далее будем проверять 
  # в каком индеке полученного list встретим словао образование

  # В задании получаемые категории указаны с маленькой буквы, и тут, 
  # мы результат приведем к такому же виду,
  # посредством метода lower
  
  # проверим, если слово образование идет под первым индексом, то вернем значение из 0-го индекса
  if ed_arr[1] == 'образование':
    return ed_arr[0].lower()
  # если под вторым индексом, то вернем склейку из первых двух элеметнов list
  elif ed_arr[2] == 'образование':
    return ' '.join(ed_arr[0:2]).lower()

    
# Создадим новый признак Образование и заполним его данными, полученными 
# в результате применения функции extract_education над признаком Оразование и Вуз
data_hh['Образование'] = data_hh['Образование и ВУЗ'].apply(extract_education)

# Удалим признак 'Образование и ВУЗ', как требуется в задании
# Так как удаляем столбец, то передаем параметр axis = 1, в дополнение, так как мы хотим 
# сделать изменения непосредствено в текущем Dataset,
# то указываем на это через параметр inplace = True
data_hh.drop(['Образование и ВУЗ'],axis = 1, inplace=True)

    Теперь нас интересует столбец "Пол, возраст". Сейчас он представлен в формате <Пол , возраст , дата рождения >. Например:

    Мужчина , 39 лет , родился 27 ноября 1979
    Женщина , 21 год , родилась 13 января 2000

Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака "Пол" и "Возраст". При этом важно учесть:

    Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина.
    Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Пол, возраст" из таблицы.

Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.

def extract_gender_age(full_info) -> tuple[str, int]:
  """Функция выделяющая пол, и приводящая к виду М или Ж, количество лет, 
  из полученной строки формата <Пол , возраст, дата рождения>
  
  Args:
      full_info (str): 'Входная строка содержащая в себе полную информацию 
                        о поле человека, возрасте и годе рождения'
  Returns:
      tuple[str, int]: Возращаем пол (допустимые значения М,Ж)), Количество лет
  """
  # преобразуем всю строку в список, которую делим по признаку ','
  all_criterias = full_info.split(',')
  
  # Значение списка all_criterias с индексом 0 содержит информацию Мужчина или Женщина, преобразуем в требуемый формат, 
  # применив условие с целью определения текста знанчения
  
  # вдруг не указан пол, вернем None
  gender = np.nan
  
  # осмотр данных показал что строка выглядит следующим образом
  # '        Мужчина ,  39 лет , родился 27 ноября 1979'
  # т.е содержит лишние пробелы до и после запятых, поэтому избавимся от них для значения пола,
  # метод strip удалит все пробелы, после этого сможем сравнить значение на предмет 
  # совпадения с Мужчина или Женщина
  if all_criterias[0].strip() == 'Мужчина':
    gender = 'М'
  elif all_criterias[0].strip() == 'Женщина':
    gender = 'Ж'
  
  # Возраст содержится в спсике all_criterias под индексом 1, и представляет собой формат <число><пробел><текстовое описание в чем измерено>. 
  # Соответсвенно мы можем также преобразовать в список, но разделив пробелом  
  # Строка с необходимым значение также содержит пробелы, однако удалить все пробелы мы не можем, как в предыдущем варианте, 
  # так как потеряем возможность разделит возраст цифрой и надпись "  39 лет " -> "39лет"
  # поэтому применим 2 метода, удаление слева и справа пробелов соответственно, и теперь разделим по пробелу
  # Вернем значение из первого индекса, что является искомым возрастом
  age = int(all_criterias[1].lstrip().rstrip().split(' ')[0])  
  
  return gender, age

# Применим нашу функцию сразу для двух ново получаемых колонок.
# Первое возвращаемое значение станет значением колонки Пол, второе же Возраст
data_hh[['Пол','Возраст']] = data_hh['Пол, возраст'].apply(lambda x: \
  pd.Series([extract_gender_age(x)[0], extract_gender_age(x)[1]]))

# Удалим признак 'Пол, возраст', как требуется в задании
# Так как удаляем столбец, то передаем параметр axis = 1, в дополнение, так как мы хотим 
# сделать изменения непосредствено в текущем Dataset,
# то указываем на это через параметр inplace = True
data_hh.drop(['Пол, возраст'],axis = 1, inplace=True)



In [ ]:


    Следующим этапом преобразуем признак "Опыт работы". Его текущий формат - это: <Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>.

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:

    Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
    Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
    В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
    В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
        Опыт работы 3 года 2 месяца…
        Опыт работы 4 года…
        Опыт работы 11 месяцев…
        Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель. Выполните преобразование, ответьте на контрольные вопросы и удалите столбец "Опыт работы" из таблицы.

def extract_month_experience(full_experience) -> int:
  """
  Вычленяем количество месяцев, которые суммарно отработал человек и 
  которые указаны в строке типа 
  <Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>.
  
  Args:
      full_experience (str): Строка содержащая полную информацию об опыте

  Returns:
      int: Сумму всех месяцев, которые человек отработал
  """
  # Создаем кортеж со всеми значениями, при которых мы должны вернуть NaN
  list_nan = (np.nan,'Не указано')
  
  if full_experience in list_nan:
    return None
  
  
  # Создаем 2 кортежа для различных возможных написаний для обозначения лет 
  # и месяцев соотвесттвенно

  list_years = ('год', 'года', 'лет')
  list_months = ('месяц', 'месяца', 'месяцев')
  
  # Делим все содержимое строки по пробелам и делаем срез по первым 6 символам
  # потому как именно описывается общий рабочий стаж
  # Пример начала строки с данными:  "Опыт работы 9 лет 9 месяцев  Февраль 2018"
  # Результат выполнения метода split: ['Опыт', 'работы', '9', 'лет', '9', 'месяцев', '', 'Февраль', '2018']
  full_experience = full_experience.split(' ')[:6]
  
  # Задаем начальные значения для месяцев и лет, как 0
  # так как в случае не нахождения одного из этих критериев нам необходимо правильно расчитать
  # общее кол-во в месяцах в конечной формуле в return
  years = 0
  months = 0
  
  # перебираем наш список и ищем вхождения указания типа времени (год, месяц)
  # если находим, то предыдущий символ будет количеством лет или месяцев соотвественно
  for index, item in enumerate (full_experience):
    if item in list_years:
      years = int(full_experience[index-1])
    if item in list_months:
      months = int(full_experience[index-1])

  # расчитываем общее колиество стажа в месяцах
  return years*12 + months

# Создаем новую колонку путем применения функции extract_month_experience к признаку 'Опыт работы'
data_hh['Опыт работы (месяц)'] = data_hh['Опыт работы'].apply(extract_month_experience)

# Удалим признак ''Опыт работы', как требуется в задании
# Так как удаляем столбец, то передаем параметр axis = 1, в дополнение, так как мы хотим 
# сделать изменения непосредствено в текущем Dataset,
# то указываем на это через параметр inplace = True
data_hh.drop(['Опыт работы'],axis = 1, inplace=True)

    Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: <Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

    Москва , не готов к переезду , готов к командировкам
    Москва , м. Беломорская , не готов к переезду, не готов к командировкам
    Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки "Город", "Готовность к переезду", "Готовность к командировкам". При этом важно учесть:

    Признак "Город" должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:

    million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.

    Признак "Готовность к переезду" должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
        … , готов к переезду , …
        … , не готова к переезду , …
        … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
        … , хочу переехать (США) , …

    Нас интересует только сам факт возможности или желания переезда.

    Признак "Готовность к командировкам" должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
        … , готов к командировкам , …
        … , готова к редким командировкам , …
        … , не готов к командировкам , …

    Нас интересует только сам факт готовности к командировке.

    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.

Выполните преобразования и удалите столбец "Город, переезд, командировки" из таблицы.

Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.

def extract_city_movements_businesstrip(full_info) -> tuple[str, bool, bool]:
  """Функция вычленяет Город, Готовности к командировкам и переездам из стоки типа 
    <Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>.
  Args:
      full_info (str): Строка, содержащая информацию с признаками «Город, переезд, командировки».

  Returns:
      tuple[str, bool, bool]: Город, Готовность к переезду, Готовность к командировкам
  """
  # Делим все содержимое строки по запятой
  full_info_splitted = full_info.split(',')
  
  # Определяем кортеж с названиями городов-милионников
  ownname_cities = ('Москва', 'Санкт-Петербург')
  million_cities = ('Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград') 
  
  # для городов реализуем следующую логику,
  # Москва и Санкт-Петербург оставляют свои названия,
  # город из списка города-милионников получает название -> город-миллионник
  # Остальные города получают значение -> другие
  # Город всегда идет первым в строке, так что работаем с первм элементом списка full_info_splitted
  
  # Инициализируем переменную в которой будем хранить вычленненое название города
  city = np.nan
  
  # Учитывая что присутствуют лишние пробелы в конце и начале каждой секции разделенной запятыми,
  # удалим пробелы слева и справа используя методы lstrip и rstrip
  # также подстрахуемся если вдруг секция с городом также содержит информацию о метро,
  # и сделаем дополнительный split с разделителем пробел и выберем первый элемент
  
  city_extracted = full_info_splitted[0].lstrip().rstrip().split(' ')[0]
  
  if city_extracted in ownname_cities:
    city = city_extracted
  elif city_extracted in million_cities:
    city = 'город-миллионник'
  else:
    city = 'другие'    
  
  # Получим информацию о желании/готовности к переезду

  #  Инициализируем переменную в которой будем полученную информацию, готов переехать челловек или нет
  # По умолчанию будем читать что готов
  ready_to_move = True

  # Проанализировав встречаемые сочетания слов, говорящих об этом мы нашли следующие сочетания, 
  # которые оформим как кортеж
  
  # Возьмем именно с предлогом не, чтобы унифицировать поиск
  ready_movements = (
    'не готов к переезду'
    ,'не готова к переезду'
  ) 
  
  # Обойдем все элементы кортежа ready_movements
  for ready_element in ready_movements:
    # приведем все к нижнему регистру и проверим для каждого элемента разделенной входной строки, начиная со второго (первый это город)
    # не входит ли данная последовательность символов ready_element в какой нибудь из сегментов входной строки
    # из возвращаемого генератора мы формируем list и проверяем его длину, если больше 0 - то найдено минимум одно вхождение
    # и значит человек не готов на переезд и мы прерываем цикл
    if len(list((el for el in full_info_splitted[1:] if ready_element.lower() in el.lower()))):
      ready_to_move = False
      break  
  
# Получим информацию о готовности к командировкам

  #  Инициализируем переменную в которой будем полученную информацию, готов переехать челловек или нет
  # исходя из условий задачи, у некоторых соискателей эта информация потерялась, и по умолчанию считаем что не готов
  
  ready_to_business_trip = False

  # Проанализировав встречаемые сочетания слов, говорящих об этом мы нашли следующие сочетания, 
  # которые оформим как 2 кортежа, те кто готовы и кто нет
  # Так как те записи, в которых мы не найдем указание на нежелание командировок, также над обудет проверить, что и отсутствует желание
  # В этом случае мы сможем точно судить, что человек не готов к командировкам
  
  # Возьмем именно с предлогом не, чтобы унифицировать поиск
  ready_business_trip = (
      'готов к командировкам'
      ,'готов к редким командировкам'
      ,'готова к командировкам'
      ,'готова к редким командировкам'
  )   
  not_ready_business_trip = (
      'не готов к командировкам'
      ,'не готова к командировкам'
  )
  
  # Введем переменную found_not_ready, которую поменяем на True, в случае нахождения информации,
  # что человек не готов к командировкам в указанной о нем инофрмации
  
  found_not_ready = False
  # Обойдем все элементы кортежа not_ready_business_trip
  for ready_element in not_ready_business_trip:
    # приведем все к нижнему регистру и проверим для каждого элемента разделенной входной строки, начиная со второго (первый это город)
    # не входит ли данная последовательность символов ready_element в какой нибудь из сегментов входной строки
    # из возвращаемого генератора мы формируем list и проверяем его длину, если больше 0 - то найдено минимум одно вхождение
    # и значит человек не готов на командировку и мы прерываем цикл
    if len(list((el for el in full_info_splitted[1:] if ready_element.lower() in el.lower()))):
      found_not_ready = True
      break  
  
  # если информации о том что соискатель не готов к командировкам нету, надо проверить, а если информация что он готов,
  # если тоже нету, то считаем что не готов
  
  if not found_not_ready:
  # Обойдем все элементы кортежа ready_business_trip
    for ready_element in ready_business_trip:
      # приведем все к нижнему регистру и проверим для каждого элемента разделенной входной строки, начиная со второго (первый это город)
      # не входит ли данная последовательность символов ready_element в какой нибудь из сегментов входной строки
      # из возвращаемого генератора мы формируем list и проверяем его длину, если больше 0 - то найдено минимум одно вхождение
      # и значит человек готов на командировку и мы прерываем цикл
      if len(list((el for el in full_info_splitted[1:] if ready_element.lower() in el.lower()))):
        ready_to_business_trip = True
        break  
    
  
  return city, ready_to_move,ready_to_business_trip


# Создаем новые 3 признака путем применения функции extract_city_movements_businesstrip к признаку 'Город, переезд, командировки'
data_hh[['Город','Готовность к переезду','Готовность к командировкам']] = data_hh['Город, переезд, командировки'].\
  apply(lambda x: pd.Series([extract_city_movements_businesstrip(x)[0],\
  extract_city_movements_businesstrip(x)[1], extract_city_movements_businesstrip(x)[2]]))

# Удалим признак 'Город, переезд, командировки', как требуется в задании
# Так как удаляем столбец, то передаем параметр axis = 1, в дополнение, так как мы хотим 
# сделать изменения непосредствено в текущем Dataset,
# то указываем на это через параметр inplace = True
data_hh.drop(['Город, переезд, командировки'],axis = 1, inplace=True)
  



In [ ]:


    Рассмотрим поближе признаки "Занятость" и "График". Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).

На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:

    полная занятость, частичная занятость
    частичная занятость, проектная работа, волонтерство
    полный день, удаленная работа
    вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

# Категории в признаках Занятость и График указаны в виде строки с разделением через запятую
# Для того, чтобы сделать One Hot Encoding, нам надо выявить какие категории находятся в каждой из строк
# Лаконично это можно сделать следующим способом: 
# 1. Преобразовать строки в список категорий
# 2. Применить метод explode, чтобы выявить какие категории относятся к каждой из строк
# 3. Применить метод pandas crosstab для ранее полученного обьекта типа Series, 
# получив из него DataFrame где столбцами станут все уникальные значения категорий, 
# а значения выставятся в зависимости от строк, True/False для категорий соответственно отих наличия
# 4. Сделать Join с текущим DataFrame

# разбиваем строку по ',' для получения списка категорий вкажой записи и возвращаем обьект list
# в новые объект типа Series для признаков Занятость и График соотвественно
employment = data_hh['Занятость'].apply(lambda x: list(x.split(',')))
work_schedule = data_hh['График'].apply(lambda x: list(x.split(',')))

# Учитывая что категории могут иметь пробелы сначала и с конца, а некоторые нет, 
# добавим приведдение к унифицированному виду, удалив пробелы с начала и конца
# Обьединим текущий DataFrame с двумя новыми, полученнымив процессе метода crosstab
# Используя фичу Python 3.7+, для удобства записи в одной строке, присвоим переменной значения колонки
# в комбинации передачи индексов (e:=employment.explode().str.lstrip().str.rstrip()).index,e
data_hh = data_hh.join(pd.crosstab((e:=employment.explode().str.lstrip().str.rstrip()).index,e))\
                 .join(pd.crosstab((ws:=work_schedule.explode().str.lstrip().str.rstrip()).index, ws))

# Удаляем более неиспользуемые признаки
data_hh.drop(['Занятость','График'],axis = 1, inplace=True)

# получим название колонок что были добавлены
added_columns = list(employment.explode().str.lstrip().str.rstrip().unique()) \
              + list(work_schedule.explode().str.lstrip().str.rstrip().unique())

# сменим тип данных для добавленных колонок на bool 
data_hh[added_columns] = data_hh[added_columns].astype('bool')



In [ ]:


    (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы "ЗП".

В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:

    30000 руб.
    50000 грн.
    550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете на платформе

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:

    "currency" - наименование валюты в ISO кодировке,
    "date" - дата,
    "proportion" - пропорция,
    "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).

Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге. Воспользуйтесь этой информацией в ваших преобразованиях.

Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке "Обновление резюме", в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования:

    Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
    Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
    Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
    Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".

# читаем данные из файла содержащим кросс курсы по дням
data_cr = pd.read_csv('data/ExchangeRates.csv', sep=',')

# Удаляем неиспользуемые признаки, зачем в памяти держать
data_cr.drop(['per','time','vol'],axis = 1, inplace=True)

# создаем словарь с сопоставлениями названий валют встречаемых в hh базе и соответствий 
# из файла с кросс курсами
ratename_relation = {
   'руб.':'RUB'
  ,'KZT':'KZT' 
  ,'USD':'USD'
  ,'бел.руб.':'BYN'
  ,'EUR':'EUR'
  ,'грн.':'UAH'
  ,'сум':'UZS'
  ,'KGS':'KGS'
  ,'AZN':'AZN'
}

# Приведем в обоих DataFrame поле содержащее дату к одному виду, обрезав time диапазон
# так как этот параметр будем использовать для связки между двумя dataFrame
data_hh['Обновление резюме'] = pd.to_datetime(data_hh['Обновление резюме'], dayfirst=True).dt.date
data_cr.date = pd.to_datetime(data_cr.date, dayfirst=True).dt.date

# Создадим 2 новых признака, куда занесем разделенные по пробелу данные зарплаты, на сумму и тип валюты соотвественно
data_hh[['ЗП (руб)','Валюта']] = data_hh['ЗП'].apply(lambda x: pd.Series([x.split(' ')[0], x.split(' ')[1]]))

# Применим метод map, чтобы переименовать данные содержащие тип валюты согласно словарю ratename_relation в ISO кодировку
# Это необходимо для второго ключевого поля по которому будем связывать данные из двух DataFrame
data_hh['Валюта'] = data_hh['Валюта'].map(ratename_relation)

# Приведем признак ЗП (руб) к типу float64, так как далее будем использовать этот признак для математических вычислений
data_hh['ЗП (руб)'] = data_hh['ЗП (руб)'].astype('float64')

# Объединим HH DataFrame с кросс курсами DataFrame
# Обьединение левого типа, так как нам необходимо получить все записи из таблицы HH, 
# и в нашем случае отсутствуют записи в таблице cross курсов для RUB, и нам не надо терять эти записи
data_hh = data_hh.merge(
  data_cr,
  how='left',
  left_on = ['Обновление резюме','Валюта'],
  right_on=['date','currency'],
)

# Выставляем значения close и proportion в 0 для записей, где валюта указана в RUB
# Можно сделать через метод fillna, но для этого надо проверить что у нас случайно не попали записи для других валют посл слияний в NaN
# например по причине отстутствия курса в требуемый день.
# Поэтому не озадачивался этим и написал изменения исключительно под записи, содержащие RUB
data_hh[['close','proportion']] = data_hh[['close','Валюта','proportion']].apply(lambda x:\
      pd.Series([
                  (1 if x['Валюта'] == 'RUB' else x.close),\
                  (1 if x['Валюта'] == 'RUB' else x.proportion)\
      ]), axis=1)

# расчитываем зарплату для всех в рублях
data_hh['ЗП (руб)'] = data_hh['ЗП (руб)'] * data_hh.close / data_hh.proportion

# Удаляем более неиспользуемые признаки
data_hh.drop(['ЗП','close','proportion','date','currency','Валюта'],axis = 1, inplace=True)



In [ ]:

Исследование зависимостей в данных

    Постройте распределение признака "Возраст". Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?

Совет: постройте гистограмму и коробчатую диаграмму рядом.

# К сожалению, px.Histogram с добавлением plotBox, некорректно работает hover_data и y(количество)
# для указаниях в всплывающих подсказках. list(н) недоступен для обьекта box, 
# а customdata для histogram
# Поменять hovertemplate по имени не можем, так как имя не можем указать при создании
# В вообщем, танцы с бубном.
# А отобразить количество сосикателей в подсказке для boxplot хочется,
# создаем новый dataFrame, где получаем количество соискателей для кадлого возраста
temp_df_counts = data_hh['Возраст'].value_counts().reset_index(name='counts')

# обьединяем предыдущий датасет с текущим обрезанным до признака Возраст,
# для получения желаемых данных
date_hh_age_count = data_hh.loc[:,['Возраст']].merge(
  temp_df_counts,
  how='inner',
  left_on=['Возраст'],
  right_on=['index'],
)


# Сделаем срез данных , получив только признак Возраст и количество. 
data_hh_age = date_hh_age_count.loc[:,('Возраст', 'counts')]

# получим модальное значение возраста сосикателей и кваннтили (для использовании в графиках)
age_modal = data_hh_age['Возраст'].mode()[0]
q25 = data_hh_age['Возраст'].quantile(0.25)
q75 = data_hh_age['Возраст'].quantile(0.75)

# Создаем график типа Гистограмма с дополнительным графиком сверху 
# BoxPlot для большей визуализации
fig = px.histogram(
  data_hh_age, 
  x="Возраст", 
  marginal="box", # указываем что нужен дополнительный график Boxplot
  nbins = 100, # зададим побольше, чтобы четко видеть моду на графике
  color_discrete_sequence=['RGB(48,128,20)'],
  hover_data=data_hh_age.columns, # Названия для label в hover для BoxPlot
  title="Распределение соискателей по признаку Возраст",
);

# Меняем стандартный hover на наш
fig.update_traces(
    hovertemplate="".join([
        "Возраст: %{x}",
        "<br>",
        "Количество: %{y}",
    ]),  
    hoverlabel_namelength = 0, # убираем вывод ховера справа, он не очень красив по мне                                             
)

# заменяем hovertemplate для box
fig.for_each_trace(
    lambda trace: trace.update(hovertemplate=trace.hovertemplate.replace\
      ("{y}","{customdata[0]}")) if trace.plotly_name == "box" else (),
)

fig.update_layout(
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100
    ),
      yaxis=dict(
        title_text="Количество соискателей",# меняем название заголовка оси ординат
        titlefont=dict(size=15), # ее размер 
    ),
    xaxis=dict(
        title_text="Возраст", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
    ), 
  paper_bgcolor="#eaf7ff",# цвет фона холста
)
# Добавим вертикальную линию для отображения моды
fig.add_vline(
  x=age_modal,
  line_width=1,
  line_dash="dash",
  line_color="black",
  annotation_text='  Модальное значение возраста соискателей:<b> ' +str(age_modal)+' лет</b>',
  row = 1, # указываем что анотация только для гистограммы
  annotation_position="top right", # позиция анотации
)
# для визуализации отразим межквантирьный размах и добавим анотации на график
fig.add_vrect(x0=q25, # для задания диапазона используем ранее полученные q25, q75
              x1=q75,
              row=1, # указываем что анотация только для гистограммы
              annotation_text=f'<b>{str(int(q25))} лет - {str(int(q75))} лет</b>',
              annotation_position="bottom left", # позиция анотации
              fillcolor="white", 
              opacity=0.4,
              line_width=0,
              annotation=dict(
                font_size=12,
                font_color = 'RGB(255,215,0)',
              )
)
fig.update_layout(hovermode="x unified")

# непосредственно вывод графика
fig.show()

Данный график нам указывает на следующие моменты:

    Мода равна 30 годам, т.е. наибольшее количество соискателей работы имеют возраст в 30 лет, таких 2834 человека
    Основное количество сосикателей в возрасте от 27 до 36 лет
    Минимальный и максимальный возраст соискателя 14 и 100 лет соотвественно
    Исходя из трудового законодательства России, Заключение трудового договора допускается с лицами, достигшими возраста шестнадцати лет, за исключением случаев, предусмотренных настоящим Кодексом, другими федеральными законами. Поэтому можно далее принять, что соискатели до 16 лет относятся к выбросам
    Если посмотреть на правую границу распределения, то все что находится за пределами 49 лет при стандартном распределении считается выбросом. Хотя я думаю стандартное распределение тут не подходит, так как много людей продолжает искать работу до пенсионного и послепенсионного возраста. Однако в данной базе количество сосикателей после 60 лет очень мало и думаю тоже должно быть позже исключено.
    Также есть один соискатель с возрастом 100 лет, я думаю это точно ошибочные данные. Насчет одного человека с возрастом в 76 лет, возможно данные корректны.
    В любом случае, скорее всего данные, где соискатели моложе 20 и старше 58 можно будет принять как выбросы. Но это посмотрим далее



In [ ]:


    Постройте распределение признака "Опыт работы (месяц)". Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?

Совет: постройте гистограмму и коробчатую диаграмму рядом.

# К сожалению, px.Histogram с добавлением plotBox, некорректно работает hover_data и y(количество)
# для указаниях в всплывающих подсказках. list(н) недоступен для обьекта box, 
# а customdata для histogram
# Поменять hovertemplate по имени не можем, так как имя не можем указать при создании
# В вообщем, танцы с бубном.
# А отобразить количество сосикателей в подсказке для boxplot хочется,
# создаем новый dataFrame, где получаем количество соискателей 
# для каждой уникальной записи Опыт работы (мес)
temp_df_exp_counts = data_hh['Опыт работы (месяц)'].value_counts().reset_index(name='counts')

# обьединяем предыдущий датасет с текущим обрезанным до признака Опыт работы (мес),
# для получения желаемых данных
data_hh_exp_count = data_hh.loc[:,['Опыт работы (месяц)']].merge(
  temp_df_exp_counts,
  how='inner',
  left_on=['Опыт работы (месяц)'],
  right_on=['index'],
)


# Сделаем срез данных , получив только признак Опыт работы (месяц)  и количество. 
data_hh_exp = data_hh_exp_count.loc[:,('Опыт работы (месяц)', 'counts')]

# получим модальное значение Опыта работы (месяц) соискателй и квантили (для использовании в графиках)
exp_modal = data_hh_exp['Опыт работы (месяц)'].mode()[0]
q25 = data_hh_exp['Опыт работы (месяц)'].quantile(0.25)
q75 = data_hh_exp['Опыт работы (месяц)'].quantile(0.75)


# Создаем график типа Гистограмма с дополнительным графиком сверху 
# BoxPlot для большей визуализации

fig = px.histogram(
  data_hh_exp, 
  x="Опыт работы (месяц)", 
  marginal="box", # указываем что нужен дополнительный график Boxplot
  nbins = 10000, # зададим побольше, чтобы четко видеть моду на графике
  color_discrete_sequence=['RGB(48,128,20)'],
  hover_data=data_hh_exp.columns, # Названия для label в hover для BoxPlot
  title="Распределение соискателей по признаку «Опыт работы (месяц)»",
);

# Меняем стандартный hover на наш
fig.update_traces(
    hovertemplate="".join([
        "Стаж в месяцах: %{x}",
        "<br>",
        "Количество соискателей: %{y}",
    ]),  
    hoverlabel_namelength = 0, # убираем вывод ховера справа, он не очень красив по мне                                             
)

# # заменяем hovertemplate для box
fig.for_each_trace(
    lambda trace: trace.update(hovertemplate=trace.hovertemplate.replace\
      ("{y}","{customdata[0]}")) if trace.plotly_name == "box" else (),
)

fig.update_layout(
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100
    ),
      yaxis=dict(
        title_text="Количество соискателей",# меняем название заголовка оси ординат
        titlefont=dict(size=15), # ее размер 
    ),
    xaxis=dict(
        title_text="Стаж в месяцах", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
        range=(0,600),
    ), 
    xaxis2=dict(
        type='log',
    ),     
  paper_bgcolor="#eaf7ff",# цвет фона холста
)
# # Добавим вертикальную линию для отображения моды
fig.add_vline(
  x=exp_modal,
  line_width=1,
  line_dash="dash",
  line_color="black",
  annotation_text='  Модальное значение рабочего стажа соискателей: в месяцах<b> ' +f'{str(int(exp_modal))}'+'</b>',
  row = 1, # указываем что анотация только для гистограммы
  annotation_position="top right", # позиция анотации
)
# для визуализации отразим межквантирьный размах и добавим анотации на график
fig.add_vrect(x0=q25, # для задания диапазона используем ранее полученные q25, q75
              x1=q75,
              row=1, # указываем что анотация только для гистограммы
              annotation_text=f'<b>{str(int(q25))} м - {str(int(q75))} м </b>',
              annotation_position="bottom left", # позиция анотации
              fillcolor="white", 
              opacity=0.4,
              line_width=0,
              annotation=dict(
                font_size=12,
                font_color = 'black',
              )
)
fig.update_layout(hovermode="x unified")

# непосредственно вывод графика
fig.show()



Для удобства визуализации я прологарифмировал по оси абсцисс для BoxPlot, для Гистограммы задал range для Гистограммы по оси абсцисс.

    Мода распределения в нашем случае составляет 81 месяц опыта(рабочего стажа)
    Значения колеблются от 1 до 1188 месяцев. В 1 месяц опыта еще можно поверить, но по любому надо детальнее проверить записи о таких людях, а вот 100 лет рабоыт верится с трудом, и явно является выбросом.
    Также надо присмотреться ко всем данным, где указан стаж работы более 300 месяцев.
    Большинство соискателей находятся в диапазоне 57 - 154 месяца опыта



In [ ]:


    Постройте распределение признака "ЗП (руб)". Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.

Совет: постройте гистограмму и коробчатую диаграмму рядом.

# К сожалению, px.Histogram с добавлением plotBox, некорректно работает hover_data и y(количество)
# для указаниях в всплывающих подсказках. list(н) недоступен для обьекта box, 
# а customdata для histogram
# Поменять hovertemplate по имени не можем, так как имя не можем указать при создании
# В вообщем, танцы с бубном.
# А отобразить количество сосикателей в подсказке для boxplot хочется,
# создаем новый dataFrame, где получаем количество соискателей для каждой уникальной записи ЗП (руб)
temp_df_salary_counts = data_hh['ЗП (руб)'].value_counts().reset_index(name='counts')

# обьединяем предыдущий датасет с текущим обрезанным до признака ЗП (руб),
# для получения желаемых данных
date_hh_salary_count = data_hh.loc[:,['ЗП (руб)']].merge(
  temp_df_salary_counts,
  how='inner',
  left_on=['ЗП (руб)'],
  right_on=['index'],
)


# Сделаем срез данных , получив только признак ЗП (руб) и количество. 
data_hh_salary = date_hh_salary_count.loc[:,('ЗП (руб)', 'counts')]

# получим модальное значение ЗП (руб) соискателей и квантили (для использовании в графиках)
salary_modal = data_hh_salary['ЗП (руб)'].mode()[0]
q25 = data_hh_salary['ЗП (руб)'].quantile(0.25)
q75 = data_hh_salary['ЗП (руб)'].quantile(0.75)


# Создаем график типа Гистограмма с дополнительным графиком сверху 
# BoxPlot для большей визуализации

fig = px.histogram(
  data_hh_salary, 
  x="ЗП (руб)", 
  marginal="box", # указываем что нужен дополнительный график Boxplot
  nbins = 10000, # зададим побольше, чтобы четко видеть моду на графике
  color_discrete_sequence=['RGB(48,128,20)'],
  hover_data=data_hh_salary.columns, # Названия для label в hover для BoxPlot
  title="Распределение соискателей по признаку Зарплата в рублях",
);

# Меняем стандартный hover на наш
fig.update_traces(
    hovertemplate="".join([
        "Зарплата в рублях: %{x}",
        "<br>",
        "Количество: %{y}",
    ]),  
    hoverlabel_namelength = 0, # убираем вывод ховера справа, он не очень красив по мне                                             
)

# # заменяем hovertemplate для box
fig.for_each_trace(
    lambda trace: trace.update(hovertemplate=trace.hovertemplate.replace\
      ("{y}","{customdata[0]}")) if trace.plotly_name == "box" else (),
)

fig.update_layout(
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100
    ),
      yaxis=dict(
        title_text="Количество соискателей",# меняем название заголовка оси ординат
        titlefont=dict(size=15), # ее размер 
        type='log'
    ),
    xaxis=dict(
        title_text="Зарплата в рублях", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
        range=(0,1000000),
    ), 
    xaxis2=dict(
        type='log'# для BoxPlot отлагарифмуем шкалу для оси x, чтобы визуализация 
        # была более читаемая
    ),     
  paper_bgcolor="#eaf7ff",# цвет фона холста
)
# # Добавим вертикальную линию для отображения моды
fig.add_vline(
  x=salary_modal,
  line_width=1,
  line_dash="dash",
  line_color="black",
  annotation_text='  Модальное значение зарплаты соискателей:<b> ' +f'{str(salary_modal):.2s}K'+' руб.</b>',
  row = 1, # указываем что анотация только для гистограммы
  annotation_position="top right", # позиция анотации
)
# для визуализации отразим межквантирьный размах и добавим анотации на график
fig.add_vrect(x0=q25, # для задания диапазона используем ранее полученные q25, q75
              x1=q75,
              row=1, # указываем что анотация только для гистограммы
              annotation_text=f'<b>{str(int(q25)):.2s}К руб - {str(int(q75)):.2s}K руб. </b>',
              annotation_position="bottom left", # позиция анотации
              fillcolor="white", 
              opacity=0.4,
              line_width=0,
              annotation=dict(
                font_size=12,
                font_color = 'black',
              )
)
fig.update_layout(hovermode="x unified")

# непосредственно вывод графика
fig.show()

Слишком большой разброс данных, как по оси абсцисс, так и ординат. Поэтому я прологарифмировал по оси абсцисс для BoxPlot и для Гистограммы по оси ординат, также я задал range для Гистограммы по оси абсцисс.

Такой вариант позволил лучше визуализировать какую ситуацию мы имеем с зарплатами.

    Мода для распределения равна 50К рублей.
    Предельные же запросы от 1 до 24М рублей, что явно указвает на некорректность таких данных. Или же указали лишь бы заполнить поле, или указали из расчета не в той валюте или еще что-нибудь
    Дипазон наиболее часто встречаемых запросов по зарплате находится в диапазоне от 37к до 95к
    Как уже указано выше,зарплаты в 1 рубль и зарплаты выше 500-600к скорее всего являются выбросами.



In [ ]:


    Постройте диаграмму, которая показывает зависимость медианной желаемой заработной платы ("ЗП (руб)") от уровня образования ("Образование"). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.

Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?

# Получаем медианную желаемую зарплату, путем группировки требуемых признаков Образование и ЗП (руб)
# для соискателей, у кого желаемая зарплата до 1М рублей
# И получения медианы для выбранных сгруппированных категорий 
data_hh_salary_experience = data_hh[['Образование','ЗП (руб)']][data_hh['ЗП (руб)'] < 1e6].\
  groupby('Образование', as_index=False).agg({'ЗП (руб)':'median'})
  
max = data_hh_salary_experience['ЗП (руб)'].max() 
data_hh_salary_experience['Percent'] = data_hh_salary_experience['ЗП (руб)'].apply(lambda x: round(x / max, 4))

fig = px.bar(
    data_hh_salary_experience,
    x='Образование',
    y='ЗП (руб)',
    hover_data = ['Percent'], #добавляем дополнительные поля для вывода в хинтах
    text='Percent', # Дополнительно указываем в каждом Баре, к какой группе относится бар,
                  # в данном графике я счиатаю это добавляет информативности 
    
)

fig.update_traces(
    marker_color=['rgb(255, 153, 51)','#54B435','RGB(30,144,255)','RGB(218,165,32)'],# задаем цвета для баров
    hoverlabel_namelength = 0, # убираем навязчивый левый надпись в ховере
    textposition='outside', # проценты пишем над баром
    texttemplate='<b style="color: #3C4048; font-size:16px;">%{customdata[0]:,.2%}</b>',
    hovertemplate="".join([
        "<b style='color: #393E46'>%{x}</b>" # вытягиваем данные из переменной указаной в hover_data[]      
        "<br>Количество: %{y}",
        "<br>",
        "Процент: %{customdata[0]:,.2%}", # вытягиваем данные из переменной указаной в hover_data[]
                                          # и форматируем чтобы проценты показывало корректно   
        
    ]),
)

fig.update_layout(
    title="Зависимость медианной желаемой заработной платы от уровня образования",
    font = dict(size=10), # уменьшим размер шрифта заголовка графика, а то не влезает
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100,
      l=100
    ),
    yaxis=dict(
        title_text="Количество соискателей", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
        range = (0,70e3), # увеличивем диапазон отражения по оси ординат, 
                          # чтобы не срезалась отрисовка процентов над левым баром        
    ),
    xaxis=dict(
        title_text="Уровень образования", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
    ), 
  paper_bgcolor="#eaf7ff",# цвет фона холста
)

fig.show()

    Наивысшая желаемая зарплата у соискателей с высшим образованием
    В наименьшей, с отрывом в 20к рублей находятся соискатели со средним и среднеспециальным образованием.
    Разброс достаточно большой в процентном соотношении, так что этот признак важен при пронозировании зароботной платы. Как можно видеть из графика, разница между высшим образованием и средним специальным отличается на 33% в среднем.



In [ ]:


    Постройте диаграмму, которая показывает распределение желаемой заработной платы ("ЗП (руб)") в зависимости от города ("Город"). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.

Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?

# Выбираем признаки Город и Желаемая зарплата для соискателей, 
# у кого желаемая зарплата до 1М рублей
data_hh_salary_city = data_hh[['Город','ЗП (руб)']][data_hh['ЗП (руб)'] < 1e6]

# Строим график типа виолончель, по оси абсцисс ЗП (руб), по оси ординат Город  
fig = px.violin(
  data_hh_salary_city, 
  x="ЗП (руб)", 
  y="Город", 
)

fig.update_traces(
    hoverlabel_namelength = 0, # убираем навязчивый левый надпись в ховере
    hovertemplate="".join([
        "<b style='color: #393E46'>Город: %{y}</b>" # вытягиваем данные из переменной указаной в hover_data[]      
        "<br>Ожидаемая зарплата: %{x}",
    ]),
    meanline_visible=True, # добавляем отображение среднее для информативности
    box_visible=True, # отображаем boxPlot внутри виолы для информативности
    line_color='mediumpurple',
    marker = dict(
      color='RGB(238,59,59)',
      line_width = 0.3,
      opacity = 0.5,
    ),
    line=dict(
      width = 0.7
    ),
    box=dict(
      fillcolor = 'lightblue',
    ), 
)

fig.update_layout(
    title="Распределение желаемой заработной платы от города<br><span style='font-size: 10px'>(для зарплат ниже 1млн рублей)</span>",
    font = dict(size=10), # уменьшим размер шрифта заголовка графика, а то не влезает
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100,
      l=100
    ),
    yaxis=dict(
        title_text="Ожидаемая зарплата", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
    ),
    xaxis=dict(
        title_text="Город", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
    ), 
  paper_bgcolor="#eaf7ff",# цвет фона холста
)

fig.show()

Ожидаемо, что в крупных городах ожидаемая зарплата выше.

    В Москве медианная желаемая зарплата 85К, против 60к в Санкт-Петербурге и 40к для двух оставшихся категорий.
    Средняя для Москвы еще выше, и составляет 108.5К, 72К Санкт-Петербург, и ниже 52К для двух оставшихся регионов.
    Разброс медианной желаемой зарплаты в зависимости от города, достаточно большой, более 30%. Соответсвенно этот признак надо учитывать при прогнозировании зарплаты



In [ ]:


    Постройте многоуровневую столбчатую диаграмму, которая показывает зависимость медианной заработной платы ("ЗП (руб)") от признаков "Готовность к переезду" и "Готовность к командировкам". Проанализируйте график, сравнив уровень заработной платы в категориях.

# простенькая функция именно под текущую выборку, без всяких проверок,
# но с идеей показать на оси абсцисс/в легенде не просто True/False, а понятные человеку сегменты
def getBooleanToText(boolValue, ind):
  if ind == 1:
      if boolValue:
          return 'Готов(а) к переезду'
      else:
          return '<b style="color:#FF3030">Не</b> готов(а) к переезду'

  elif ind == 2:
      if boolValue:
          return 'Готов(а) к командировкам'
      else:
          return '<b style="color:#FF3030">Не</b> готов(а) к командировкам'

# Получим требуемый DataFrame, а именно сгрупированный по Готовности к командировкам и переездам 
# с вычислением медианной желаемой зарплаты

data_hh_ready_to_move = data_hh[['Готовность к переезду','Готовность к командировкам','ЗП (руб)']].\
  groupby(['Готовность к переезду','Готовность к командировкам'], as_index=False).agg({'ЗП (руб)':'median'})

# Заменим True и False в humanlanguage описание
data_hh_ready_to_move[['Готовность к переезду','Готовность к командировкам']] = data_hh_ready_to_move[['Готовность к переезду','Готовность к командировкам']].\
  apply(lambda x: pd.Series([\
    getBooleanToText(x['Готовность к переезду'],1),\
    getBooleanToText(x['Готовность к командировкам'],2),
  ]), axis = 1)


fig = px.bar(
    data_hh_ready_to_move,
    x='Готовность к переезду',
    y='ЗП (руб)', # Готовность к командировкам
    color="Готовность к командировкам", # Разобьем по категрии по Готовности к командировкам
    color_discrete_sequence =['RGB(205,170,125)','RGB(100,149,237)'],
    barmode='group', # поставим колонки рядом для каждого из сегмента по оси абсцисс 
    hover_data = ['Готовность к командировкам'], #добавляем дополнительные поля для вывода в хинтах
    text='Готовность к командировкам', # Дополнительно указываем в кажом Баре, к какой группе относится бар,
                  # в данном графике я счиатю это очень полезно   
)
fig.update_layout(hovermode="x unified") # Указываем что подсказку показываем сразу для всех элементов 
                                         # в разрезе по оси X.

# Меняем стандартный hover на наш
fig.update_traces(
    textposition='inside',
    texttemplate='<span style="color:#1E1E1E">     %{customdata[0]}</span>',
    hovertemplate="".join([
        "<b style='color: #20293d'>%{customdata[0]}</b>" # вытягиваем данные из переменной указаной в hover_data[]      
        "<br>Медианная зарплата: %{y}",
    ]),
    hoverlabel_namelength = 0, # убираем вывод ховера сверху,
                               # потому как хочу убрать двоеточие в стандартном выводе и добавить стиль bold
)

fig.update_layout(
    title="Зависимость медианной желаемой зарплаты от готовности к командировкам/переездам",
    font = dict(size=10), # уменьшим размер шрифта заголовка графика, а то не влезает
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100,
      l=100
    ),
    legend=dict(# играемся с параметрами легенды
        borderwidth = .5,# ширина рамки в полпикселя
        bordercolor = '#cbd7de',# цвет рамки
        bgcolor = '#f2faff',# цвет фона
        orientation="h", # показать сверху
        yanchor="bottom",# вертикальное выравнивание по нижней части легенды относительно холста
        y=1.01, # сдвигаем на 1% выше, чтобы не сливалась нижняя граница рамки легенды с графиком
        xanchor="right", # выравнивание блока легенды по правому краю
        x=1, # никаких сдвигов не надо, будет на уровне с графиком по правому краю
        title_text="", # убираем title для легенды, в данном случае понятно без лишних текстов
    ),     
    yaxis=dict(
        titlefont=dict(size=15),  # ее размер 
        title_text='Медианная ЗП (руб)'
    ),
    xaxis=dict(
        titlefont=dict(size=15),  # ее размер 
    ), 
  paper_bgcolor="#eaf7ff",# цвет фона холста
)
# Добавим на каждом баре дополнительную информацию о готовности к командировкам,
# легче ориентироваться в графике, какой бар к кому относится
fig.update_traces(textfont_size=10, textangle=90, textposition="inside")
fig.show()

Самые высокие требования к зарплате у соискателей, которые готовы на все ради работы мечты, и на командировки и на переезд.

    Разница между тотальными домоседами и готовыми на любые движения: 40к против 66к в медианной зарплате.
    По сегментно можно видеть, что люди не готовые на переезд, но готовые к командировкам, ожидают в примерно 33% больше зарплату. 40к против 60к. В сегменте готовых к переезду, разница ниже между готовыми к командировкам и нет: 50к против 65к, т.е 25%.
    Отсюда можно сделать вывод, что данные оба признака являются важными в прогнозировании уровня желаемой зарплаты, и что готовность к командировкам повышает желание увеличение зарплаты в среднем на 25-30%



In [ ]:


    Постройте сводную таблицу, иллюстрирующую зависимость медианной желаемой заработной платы от возраста ("Возраст") и образования ("Образование"). На полученной сводной таблице постройте тепловую карту. Проанализируйте тепловую карту, сравнив показатели внутри групп.

# Создадим сводную таблицу по требуемым полям, NaN значения при этом, заполним нулями
data_hh_age_exp_salary = data_hh[['Возраст','Образование','ЗП (руб)']].\
    pivot_table(index='Образование',columns='Возраст',values='ЗП (руб)',\
        aggfunc='median',fill_value=0)

fig = px.imshow(
    data_hh_age_exp_salary,
    text_auto=".2%",
    aspect="auto",
    color_continuous_scale  = 'darkmint',
)


fig.update_traces(
    hovertemplate="".join([
        "<b> Возраст: %{x}</b>"       
        "<br>Возраст: %{y}",        
        "<br>Ожидаемая медианая зарплата: %{z}",
    ]),
    hoverlabel_namelength = 0, # убираем навязчивый левый надпысь в ховере
)

fig.update_layout(
    title="Зависимость медианной желаемой заработной платы от возраста и образования",
    font = dict(size=10), # уменьшим размер шрифта заголовка графика, а то не влезает
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100,
      l=30
    ),
    coloraxis_colorbar=dict(
        title="Желаемая медианая зарплата", # меняем заголовок для цветного бара справа
        title_side="right", # разместим надпись справа
    ), 
    yaxis=dict(
        titlefont=dict(size=15),  # ее размер 
    ),
    xaxis=dict(
        title_text="Возраст соискателя в годах", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер
        nticks=30 
    ),    
  paper_bgcolor="#eaf7ff",# цвет фона холста
)

fig.show()

Если не обращать внимание на выбросы, четко проглядывается на тепловом графике тенденция, что с увеличением возраста соискателя в группах "высшее" и "неоконченное высшее" происходит достаточно "агрессивный" рост желаемой медианной зарплаты. В сегментах "среднее" и "среднее специальное" образование, данный рост минимален.

Также интересное замечание, что людей с высшим образованием можно встретить аж до 90 лет, тогда как люди со средним оразованием, после 60-ти уже нету.

Для среднего и средне специального, параметр возраст не имеет особой важности при прогнозировании зарплаты, тогда как для высшего и высшего незаконченного, это по праву важный параметр.


In [ ]:


    Постройте диаграмму рассеяния, показывающую зависимость опыта работы ("Опыт работы (месяц)") от возраста ("Возраст"). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

# Сделаем срез данных , получив только столбцы Опыт работы в месяцах и Возраст. 
data_hh_exp_age = data_hh.loc[:,['Опыт работы (месяц)','Возраст']]

# Переведем Опыт работы в месяцах в года. 
data_hh_exp_age['Опыт работы (лет)'] = data_hh_exp_age['Опыт работы (месяц)'].apply(lambda \
  x: x / 12)


fig = go.Figure()
fig.add_trace(
  go.Scatter(
    x=data_hh_exp_age['Возраст'],
    y=data_hh_exp_age['Опыт работы (лет)'],
    mode="markers",
    marker_color='RGB(30,144,255)',
))

# Отрисуем референсную линию, помогающая легче определить когда опыт в годах выше кол-во лет соискателя
fig.add_shape(type="line",
    x0=0, y0=0, x1=100, y1=100,
    line=dict(
        color="RGB(238,18,137)",
        width=1,
        dash="dot",
    )
)

# Добавим аннотацию на график, поясняющая что за линия нарисована
fig.add_annotation(
    x=70, y=70,
    text="Граница, когда опыт равен годам жизни",
    showarrow=True,
    arrowhead=3,
    align = "left",
    arrowcolor="RGB(238,18,137)",
    
)


fig.update_layout(
    title="Зависимость опыта работы в годах от возраста",
    font = dict(size=10), # уменьшим размер шрифта заголовка графика, а то не влезает
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100,
      l=80
    ),
    yaxis=dict(
        title_text="Опыт работы соискателя в годах", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
    ),
    xaxis=dict(
        title_text="Возраст соискателя в годах", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
    ), 
  paper_bgcolor="#eaf7ff",# цвет фона холста
)

# непосредственно вывод графика
fig.show()

Данный график отображает взаимосвязь опыта работы в годах от возраста соискателя.

Как мы видим, у 7 человек (2 точки лежат почти одна на одной) указали, что имеют рабочий стаж, превышающий их возраст. Такое в принципе возможно, если учест, что человек одновременно работал на нескольких работах и считал стаж паралельно, но в нашем случае это аномальные данные, и такие данные из прогноза должны быть вычеркнуты.


In [ ]:


Дополнительные баллы

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".

Дополнительно проверим, как влияет желание работать удаленно на желаемую зарплату.

# простенькая функция именно под текущую выборку, без всяких проверок,
# но с идеей показать на оси абсцисс/в легенде не просто True/False, а понятные человеку сегменты
def getBooleanRemoteToText(boolValue):
  if boolValue:
    return 'Удаленная работа'
  else:
    return 'Работа в офисе'

# Получаем медианную желаемую зарплату, путем группировки требуемых признаков Образование и ЗП (руб)
# для соискателей, у кого желаемая зарплата до 1М рублей (срезаем выбросы)
# И получения медианы для выбранных сгруппированных категорий 
data_hh_salary_remote = data_hh[['удаленная работа','ЗП (руб)']][data_hh['ЗП (руб)'] < 1e6].\
  groupby('удаленная работа', as_index=False).agg({'ЗП (руб)':'median'})
  
max = data_hh_salary_remote['ЗП (руб)'].max() 
data_hh_salary_remote['Percent'] = data_hh_salary_remote['ЗП (руб)'].apply(lambda x: round(x / max, 4))

data_hh_salary_remote['удаленная работа'] = data_hh_salary_remote['удаленная работа'].apply(\
  lambda x: getBooleanRemoteToText(x))

fig = px.bar(
    data_hh_salary_remote,
    x='удаленная работа',
    y='ЗП (руб)',
    hover_data = ['Percent'], #добавляем дополнительные поля для вывода в хинтах
    text='Percent', # Дополнительно указываем в каждом Баре, к какой группе относится бар,
                  # в данном графике я счиатаю это добавляет информативности 
    
)

fig.update_traces(
    marker_color=['rgb(255, 153, 51)','#54B435','RGB(30,144,255)','RGB(218,165,32)'],# задаем цвета для баров
    hoverlabel_namelength = 0, # убираем навязчивый левый надпись в ховере
    textposition='outside', # проценты пишем над баром
    texttemplate='<b style="color: #3C4048; font-size:16px;">%{customdata[0]:,.2%}</b>',
    hovertemplate="".join([
        "<b style='color: #393E46'>%{x}</b>" # вытягиваем данные из переменной указаной в hover_data[]      
        "<br>Медианная желаемая зарплата: %{y}",
        "<br>",
        "Процент: %{customdata[0]:,.2%}", # вытягиваем данные из переменной указаной в hover_data[]
                                          # и форматируем чтобы проценты показывало корректно   
        
    ]),
)

fig.update_layout(
    title="Зависимость медианной желаемой заработной платы от уровня образования",
    font = dict(size=10), # уменьшим размер шрифта заголовка графика, а то не влезает
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100,
      l=100
    ),
    yaxis=dict(
        title_text="Желаемая медианая заплата в руб.", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
        range = (0,70e3), # увеличивем диапазон отражения по оси ординат, 
                          # чтобы не срезалась отрисовка процентов над левым баром        
    ),
    xaxis=dict(
        title_text="", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
    ), 
  paper_bgcolor="#eaf7ff",# цвет фона холста
)

fig.show()

На удивление, соисткатели предпочитающие удаленный стиль работы, хотят более высокую зарплату, примерно на 10%

# Выбираем признаки Город и Желаемая зарплата для соискателей, 
# у кого желаемая зарплата до 1М рублей
data_hh_salary_city_remote = data_hh[['Город','удаленная работа','ЗП (руб)']][data_hh['ЗП (руб)'] < 1e6]

data_hh_salary_city_remote['удаленная работа'] = data_hh_salary_city_remote['удаленная работа'].apply(\
  lambda x: getBooleanRemoteToText(x))

# Строим график типа виолончель, по оси абсцисс ЗП (руб), по оси ординат Город  
fig = px.violin(
  data_hh_salary_city_remote, 
  x="ЗП (руб)", 
  y="Город",
  color='удаленная работа',
  color_discrete_sequence =['RGB(205,170,125)','RGB(100,149,237)'], 
)

fig.update_traces(
    hoverlabel_namelength = 0, # убираем навязчивый левый надпись в ховере
    hovertemplate="".join([
        "<b style='color: #393E46'>Город: %{y}</b>" # вытягиваем данные из переменной указаной в hover_data[]      
        "<br>Ожидаемая зарплата: %{x}",
    ]),
    meanline_visible=True, # добавляем отображение среднее для информативности
    box_visible=True, # отображаем boxPlot внутри виолы для информативности
    marker = dict(
      line_width = 0.3,
      opacity = 0.5,
    ),
    line=dict(
      width = 0.7
    ),
    box=dict(
      fillcolor = 'lightblue',
    ), 
)

fig.update_layout(
    title="Распределение желаемой заработной платы от города от типа работы<br><span style='font-size: 10px'>(для зарплат ниже 1млн рублей)</span>",
    font = dict(size=10), # уменьшим размер шрифта заголовка графика, а то не влезает
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100,
      l=100
    ),
    legend = dict(
      title_text = '',
      borderwidth = .5,# ширина рамки в полпикселя
      bordercolor = '#cbd7de',# цвет рамки
      bgcolor = '#f2faff',# цвет фона
      orientation="h", # показать сверху
      yanchor="bottom",# вертикальное выравнивание по нижней части легенды относительно холста
      y=1.01, # сдвигаем на 1% выше, чтобы не сливалась нижняя граница рамки легенды с графиком
      xanchor="right", # выравнивание блока легенды по правому краю
      x=1, # никаких сдвигов не надо, будет на уровне с графиком по правому краю      
    ),
    yaxis=dict(
        title_text="Ожидаемая зарплата", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
    ),
    xaxis=dict(
        title_text="Город", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
    ), 
  paper_bgcolor="#eaf7ff",# цвет фона холста
)

fig.show()

Как мы видим, ожидаемая медианная зарплата только в Санкт-Петербурге, что в случае удаленной работы, что для посещения офиса, находиьтся на одном уровне.

Для других регионов, разница составляет примерно 9-12%, выше для удаленной работы.

В зависимости оттого, насколько точным мы хотим сделать прогноз предполагаемой зарплаты, эту зависимость можно использовать, но скажем так, только в случае построения модели, дающе более точные прогнозы.


In [ ]:

Очистка данных

    Начнем с дубликатов в наших данных. Найдите полные дубликаты в таблице с резюме и удалите их.

# проверяем на дубликаты, в нашем случае полностью уникадьные строки
data_hh.duplicated().sum()
# удаляем дубликаты
data_hh = data_hh.drop_duplicates()

    Займемся пропусками. Выведите информацию о числе пропусков в столбцах.

# Выведем колонки, где имеются пропуски и количество этих пропусков
display(data_hh.isnull().sum()[data_hh.isnull().sum() > 0])

Последнее/нынешнее место работы      1
Последняя/нынешняя должность         2
Опыт работы (месяц)                168
dtype: int64

    Итак, у нас есть пропуски в 3ех столбцах: "Опыт работы (месяц)", "Последнее/нынешнее место работы", "Последняя/нынешняя должность". Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните медианным значением.

# Удалим строки, где хотя бы есть 1 пропуск в признаках Последнее/нынешнее место работы,Последняя/нынешняя должность
data_hh=data_hh[~(data_hh['Последнее/нынешнее место работы'].isnull() | data_hh['Последняя/нынешняя должность'].isnull())]


# Заменим пропуски в признаке Опыт работы в месяцах, медианным значением
values = {
    'Опыт работы (месяц)': data_hh['Опыт работы (месяц)'].median(),
}
#заполняем пропуски в соответствии с заявленным словарем
data_hh = data_hh.fillna(values)

    Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

# Удалим данные, где зарплата менее 1000 или более 1М
data_hh = data_hh[~((data_hh['ЗП (руб)'] > 1e6) | (data_hh['ЗП (руб)'] < 1e3))]

    В процессе разведывательного анализа мы обнаружили резюме, в которых опыт работы в годах превышал возраст соискателя. Найдите такие резюме и удалите их из данных

# Удалим данные, где Опыт работы в годах выше Возраста соискателя
data_hh = data_hh[~(data_hh['Опыт работы (месяц)']/12 > data_hh['Возраст'])]

    В результате анализа мы обнаружили потенциальные выбросы в признаке "Возраст". Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в логарифмическом масштабе. Добавьте к графику линии, отображающие среднее и границы интервала метода трех сигм. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

histplot.axvline(log_age.mean(), color='k', lw=2)

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику. Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на 1 сигму (возьмите 4 сигмы) в правую сторону.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

# Сделаем срез данных , получив только признак Возраст 
data_hh_age_log = np.log(date_hh_age_count['Возраст']).to_frame()

# получим модальное значение возраста сосикателей и левую и правые сигмы (для использовании в графиках)
age_modal_log = data_hh_age_log['Возраст'].mode()[0]
lsigma = data_hh_age_log['Возраст'].mean() - 3 * data_hh_age_log['Возраст'].std()
rsigma = data_hh_age_log['Возраст'].mean() + 3 * data_hh_age_log['Возраст'].std()

# Создаем график типа Гистограмма с дополнительным графиком сверху 
# BoxPlot для большей визуализации
fig = px.histogram(
  data_hh_age_log, 
  x="Возраст", 
  nbins = 100, # зададим побольше, чтобы четко видеть моду на графике
  color_discrete_sequence=['RGB(48,128,20)'],
  # hover_data=data_hh_age.columns, # Названия для label в hover для BoxPlot
  title='Распределение соискателей по признаку Возраст <br>\
<span style="font-size: 10px">(по логарифмической шкале)',
)

# Меняем стандартный hover на наш
fig.update_traces(
    hovertemplate="".join([
        "Возраст: %{x}",
        "<br>",
        "Количество: %{y}",
    ]),  
    hoverlabel_namelength = 0, # убираем вывод ховера справа, он не очень красив по мне                                             
)

fig.update_layout(
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100
    ),
      yaxis=dict(
        title_text="Количество соискателей",# меняем название заголовка оси ординат
        titlefont=dict(size=15), # ее размер 
    ),
    xaxis=dict(
        title_text="Возраст по логарифмической шкале", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
    ), 
  paper_bgcolor="#eaf7ff",# цвет фона холста
)
# Добавим вертикальную линию для отображения моды
fig.add_vline(
  x=age_modal_log,
  line_width=1,
  line_dash="dash",
  line_color="black",
  annotation_text='  Модальное log возраста соискателей:<b> ' +f'{str(age_modal_log):.4s}'+'</b>', 
  row = 1, # указываем что анотация только для гистограммы
  annotation_position="top right", # позиция анотации
)

# Добавим вертикальную линию для отображения моды
fig.add_vline(
  x=lsigma,
  line_width=1,
  line_dash="dot",
  line_color="violet",
  annotation_text=f'<b>Левая Sigma {str(lsigma):.4s}',
  annotation_position="top right", # позиция анотации
)

# Добавим вертикальную линию для отображения моды
fig.add_vline(
  x=rsigma,
  line_width=1,
  line_dash="dot",
  line_color="violet",
  annotation_text=f'<b>Правая Sigma {str(rsigma):.4s}',
  annotation_position="bottom right", # позиция анотации
)
# для визуализации отразим межквантирьный размах и добавим анотации на график
fig.add_vrect(x0=lsigma, # для задания диапазона используем ранее полученные q25, q75
              x1=rsigma,
              row=1, # указываем что анотация только для гистограммы
              fillcolor="white", 
              opacity=0.4,
              line_width=0,
)
fig.update_layout(hovermode="x unified")

# непосредственно вывод графика
fig.show()
fig.write_html("plotly_graphs/data_hh_age_log.html")

Ассиметрично логарифмическое распределение в нашем случае является правосторонним.


In [ ]:
# Функция с реализацией z-отклонения
def outliers_z_score(data, feature, log_scale=False, left = 3, right = 3):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
  
outliers, cleaned = outliers_z_score(data_hh, 'Возраст', log_scale=True, left = 3, right = 3)

print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 112
Результирующее число записей: 44373

В случае стандартного применения функции поиска z-отклонений для выявления выбросов, т.е. при правой и левой сигме равной 3-ём, 112 записей по признаку Возраст считаются выбросами

outliers['Возраст'].value_counts(ascending=True).sort_index(ascending=True)

Как мы видим из вывода статистики сверху, в выбросы попали сосикатели 15 лет и выше 62 лет. Основное количество соискатеелй в возрасте от 63(включительно) до 66 лет(включительно)



In [ ]:
В случае если мы сделаем послабление, и сдвинем правую сигму на 1 единицу правее, т.е на 4 единицы вместо 3-х, количество выбросов уменьшится до 3. И туда попадут соискатели 15 лет и 100 лет.

outliers, cleaned = outliers_z_score(data_hh, 'Возраст', log_scale=True, left = 3, right = 4)

print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')
outliers['Возраст'].value_counts(ascending=True).sort_index(ascending=True)

Число выбросов по методу z-отклонения: 3
Результирующее число записей: 44482

15     2
100    1
Name: Возраст, dtype: int64

display(outliers)



In [ ]:
# Удалим из данных соискателей, который попали под выбросы 3л,4пр сигм

outliers, cleaned = outliers_z_score(data_hh, 'Возраст', log_scale=True, left = 3, right = 4)

data_hh = cleaned

